In [1]:
import pandas as pd
import requests

In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

# Access the key
API_KEY = os.getenv("FBR_API_KEY")

if not API_KEY:
    raise ValueError("⚠️ Missing FBR_API_KEY in your .env file!")

print("✅ API key loaded successfully!")

headers = {"X-API-Key": API_KEY}

✅ API key loaded successfully!


In [3]:
headers = {"X-API-Key": API_KEY}
response = requests.get("https://fbrapi.com/leagues", headers=headers)
print(response.status_code)

ConnectionError: HTTPSConnectionPool(host='fbrapi.com', port=443): Max retries exceeded with url: /leagues (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x10f0ff4d0>: Failed to resolve 'fbrapi.com' ([Errno 8] nodename nor servname provided, or not known)"))

In [4]:
# Test whether the API key works
url = "https://fbrapi.com/player-match-stats"
params = {
    "player_id": "92e7e919",
    "league_id": "9",
    "season_id": "2023-2024"
}
headers = {"X-API-Key": API_KEY}

response = requests.get(url, params=params, headers=headers)
print(response.status_code)

ConnectionError: HTTPSConnectionPool(host='fbrapi.com', port=443): Max retries exceeded with url: /player-match-stats?player_id=92e7e919&league_id=9&season_id=2023-2024 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x12f292c10>: Failed to resolve 'fbrapi.com' ([Errno 8] nodename nor servname provided, or not known)"))

In [5]:
# Not very useful. Data from teams endpoint is better.

response = requests.get("https://fbrapi.com/matches", params={"league_id": 9, "season_id": "2024-2025"}, headers=headers)
data = response.json()
records = data['data']
matches = pd.json_normalize(records)

ConnectionError: HTTPSConnectionPool(host='fbrapi.com', port=443): Max retries exceeded with url: /matches?league_id=9&season_id=2024-2025 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x12f5887d0>: Failed to resolve 'fbrapi.com' ([Errno 8] nodename nor servname provided, or not known)"))

In [ ]:
# Get list of Premier League teams and save to CSV
#tms = matches[['home', 'home_team_id']].drop_duplicates(subset=['home', 'home_team_id']).reset_index(drop=True)
# teams = teams.rename(columns={'team': 'home', 'team_id': 'home_team_id'})
# teams = pd.concat([teams, tms], ignore_index=True, axis=0).drop_duplicates(subset=['home_team_id']).reset_index(drop=True)
# teams.to_csv('../data/pl_teams.csv', index=False)

In [5]:
# Load teams from CSV and create a dictionary mapping team names to IDs
teams = pd.read_csv('../data/pl_teams.csv')
tms = teams['home'].unique()
ids = teams['home_team_id'].unique()

teams = {}
for i in range(len(tms)):
    teams[tms[i]] = ids[i]
print(teams)

{'Aston Villa': '8602292d', 'Wolves': '8cec06e1', 'Manchester City': 'b8fd03ef', 'Newcastle Utd': 'b2b47a98', 'Brentford': 'cd051869', 'Crystal Palace': '47c64c55', "Nott'ham Forest": 'e4a775cb', 'Burnley': '943e8050', 'Everton': 'd3fd31cc', 'Brighton': 'd07537b9', 'Chelsea': 'cff3d9bb', 'West Ham': '7c21e445', 'Liverpool': '822bd0ba', 'Leeds United': '5bfb9659', 'Sunderland': '8ef52968', 'Tottenham': '361ca564', 'Arsenal': '18bb7c10', 'Manchester Utd': '19538871', 'Fulham': 'fd962109', 'Bournemouth': '4ba7cbea', 'Sheffield Utd': '1df6b87e', 'Luton Town': 'e297cd13', 'Ipswich Town': 'b74092de', 'Leicester City': 'a2d435b3', 'Southampton': '33c895d4'}


In [6]:
# Gather matches from each team
import time
import random

pl_matches = pd.DataFrame()

for team in teams.keys():
    response = requests.get("https://fbrapi.com/teams", params={"team_id": teams[team], "season_id": "2024-2025"}, headers=headers)
    data = response.json()
    records = data['team_schedule']['data']
    df = pd.json_normalize(records)
    df['home_team'] = team
    df['home_team_id'] = teams[team]
    pl_matches = pd.concat([pl_matches, df[df['league_id'] == 9]], ignore_index=True)
    time.sleep(random.uniform(3, 7)) # Sleep between 3 to 7 seconds to avoid rate limiting

KeyError: 'team_schedule'

In [ ]:
pl_matches['date'] = pd.to_datetime(pl_matches['date'])
pl_matches = pl_matches[pl_matches['date'] <= pd.Timestamp.now()].sort_values(by='date', ascending=False)
pl_matches.reset_index(drop=True, inplace=True)
pl_matches

,date,time,match_id,league_name,league_id,opponent,opponent_id,home_away,result,gf,ga,attendance,captain,formation,referee,home_team,home_team_id
0,2025-10-05,16:30,8c0ee359,Premier League,9,Brentford,cd051869,Away,W,1.0,0.0,"17,028",Rúben Dias,4-1-4-1,Darren England,Manchester City,b8fd03ef
1,2025-10-05,14:00,db870461,Premier League,9,Aston Villa,8602292d,Away,L,1.0,2.0,"40,308",Josh Cullen,5-4-1,Andrew Kitchen,Burnley,943e8050
2,2025-10-05,14:00,88e53add,Premier League,9,Crystal Palace,47c64c55,Home,W,2.0,1.0,"51,770",James Tarkowski,4-2-3-1,Michael Salisbury,Everton,d3fd31cc
3,2025-10-05,14:00,1314f9fd,Premier League,9,Brighton,d07537b9,Home,D,1.0,1.0,"29,504",João Gomes,4-3-3,Jarred Gillett,Wolves,8cec06e1
4,2025-10-05,14:00,1314f9fd,Premier League,9,Wolves,8cec06e1,Away,D,1.0,1.0,"29,504",Lewis Dunk,4-2-3-1,Jarred Gillett,Brighton,d07537b9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1655,2023-08-12,15:00,56a137f7,Premier League,9,Brighton,d07537b9,Away,L,1.0,4.0,"31,872",Tom Lockyer,3-5-2,David Coote,Luton Town,e297cd13
1656,2023-08-12,15:00,55fd92c7,Premier League,9,Sheffield Utd,1df6b87e,Away,W,1.0,0.0,"31,194",Joel Ward,4-2-3-1,John Brooks,Crystal Palace,47c64c55
1657,2023-08-12,15:00,15addfc7,Premier League,9,Fulham,fd962109,Home,L,0.0,1.0,"39,940",James Tarkowski,4-4-1-1,Stuart Attwell,Everton,d3fd31cc
1658,2023-08-11,20:00,3a6836b4,Premier League,9,Burnley,943e8050,Away,W,3.0,0.0,"21,572",Kevin De Bruyne,4-2-3-1,Craig Pawson,Manchester City,b8fd03ef


In [28]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd

url = "https://fbref.com/en/comps/9/Premier-League-Stats"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/131.0.0.0 Safari/537.36"
    ),
    "Accept": (
        "text/html,application/xhtml+xml,application/xml;"
        "q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8"
    ),
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-User": "?1",
    "Sec-Fetch-Dest": "document",
    "Upgrade-Insecure-Requests": "1",
    "Cache-Control": "no-cache",
    "Pragma": "no-cache",
    "sec-ch-ua": '"Chromium";v="131", "Not_A Brand";v="24", "Google Chrome";v="131"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
}


s = requests.Session()
response = s.get(url, headers=headers)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")

# FBref hides real tables inside HTML comments
comments = soup.find_all(string=lambda text: isinstance(text, Comment))

teams_table_html = None

for c in comments:
    if 'stats_squads_standard_for' in c:
        teams_table_html = c
        break

# Parse the commented-out table
table_soup = BeautifulSoup(teams_table_html, "html.parser")
table = table_soup.find("table")

# Extract headers
headers = [th.get_text(strip=True) for th in table.find("thead").find_all("th")]

# Extract body rows
rows = []
for tr in table.find("tbody").find_all("tr"):
    cells = [td.get_text(strip=True) for td in tr.find_all(["th", "td"])]
    rows.append(cells)

df = pd.DataFrame(rows, columns=headers)

df.head()

TypeError: Incoming markup is of an invalid type: None. Markup must be a string, a bytestring, or an open filehandle.

In [ ]:
pl_matches.to_csv('../data/pl_matches.csv', index=False)